### WBS Coding School
___
### --PROJECT--
# Supervised Machine Learning
### :: Regression: Housing prices

- *01: Data Preparation and Exploration*
- *02: Baseline Model*
- *03: Feature Selection*
- **04: Model Training and Selection**

___

# Model Training and Selection
___

In this script, we will deploy and compare a variety of different classification models and find each model's best set of hyper parameters.

## Table of contents:
- [1 Models](#models)
    - [1.1 Lazy Predict](#lazy)
    - [1.2 XGB Regressor](#tree)
    - [1.3 Gradient Boosting Regressor](#forest)
    - [1.4 Poisson Regressor](#poisson)
- [2 Model Selection](#selection)
    - [2.1 Model Scores](#scores)
    - [2.2 Voting Regressor](#voting)
- [3 Make Prediction](#prediction)

#### Import Libraries, Data & Preprocessing Pipeline

In [1]:
import pandas as pd
import joblib

from lazypredict.Supervised import LazyRegressor

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV

from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import PoissonRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, mean_squared_log_error

___
<a id="selection"></a>
# 2&nbsp; Model Selection

In [2]:
# Import the cleaned data
housing_data = pd.read_csv("data/train_clean.csv")

# Import the preprocessing pipeline
preprocessing_pipe = joblib.load('pipelines/preprocessing_pipe.joblib')

#### Splitting

In [3]:
# X and y creation
X = housing_data
y = X.pop("SalePrice")

# Data splitting
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

___
<a id="models"></a>
# 1&nbsp; Models

Here we'll first do a pre-selection of models using `lazypredict`, then we'll tune each of the best performing models one after another.

But first, let's first initiate a dictionary in which we can store model scores for comparison and create two little helper functions for printing model performance.


##### Note:
For two reasons we're not using feature selectors such as the `RFECV` here.
- First of all, runtimes go through the roof if we do.
- Secondly, we did not find model performance to increase using `RFEC` or `ExtraTreesRegressor` selectors.

In [4]:
# Initiate dictionary for model scores
model_scores = pd.DataFrame(
    columns=["R2", "MAE", "MAPE", "MSE", "RMSE_log"]
)

In [5]:
def get_model_metrics(model_name: str, y_true, y_pred):
    metrics = pd.DataFrame({
        "R2": r2_score(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "MAPE": mean_absolute_percentage_error(y_true, y_pred),
        "MSE": mean_squared_error(y_true, y_pred),
        "RMSE_log": mean_squared_log_error(y_true, y_pred)
    }, index=[model_name])
    return metrics

def print_best_parameters(grid_search_pipeline, X, y):
    print("\nRMSE_log: ", mean_squared_log_error(grid_search_pipeline.predict(X), y))
    print("Best logreg parameters: \n", grid_search_pipeline.best_params_)

<a id="lazy"></a>
### 1.1&nbsp; Lazy Predict

We'll use `lazypredict` to find the set of best performing models for our task.

In [6]:
lazy_regressor = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
models, predictions = lazy_regressor.fit(X_train, X_test, y_train, y_test)

  0%|          | 0/42 [00:00<?, ?it/s]

 98%|█████████▊| 41/42 [01:05<00:01,  1.55s/it]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002273 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3220
[LightGBM] [Info] Number of data points in the train set: 1168, number of used features: 162
[LightGBM] [Info] Start training from score 181441.541952


100%|██████████| 42/42 [01:05<00:00,  1.56s/it]


In [7]:
# The five models with the lowest RMSE:
models.nsmallest(5, columns="RMSE")

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
XGBRegressor,0.88,0.92,25516.91,0.36
GradientBoostingRegressor,0.87,0.90,27175.09,0.62
PoissonRegressor,0.86,0.90,28331.29,0.25
RandomForestRegressor,0.85,0.89,28541.71,1.51
HistGradientBoostingRegressor,0.85,0.89,28592.04,0.65


The best 3 models are XGBoosted Trees, Gradient Boosting and a Poisson Model. Each of them has an R^2 >= 0.9. 

Let's see if we can further tune their performance.

<a id="xgb"></a>
### 1.2&nbsp; XGB Regressor

The kaggle challenge requires the root mean squared log, therefore we use `scoring="neg_mean_squared_log_error"` for the Randomized Search.

In [8]:
# Make pipeline
xgb_pipeline = make_pipeline(
    preprocessing_pipe,
    XGBRegressor()
)

# Perform hyper parameter search
param_grid_xgb = {
    "xgbregressor__n_estimators": [50, 100, 150, 200],
    "xgbregressor__max_depth": range(3, 12, 2),
    "xgbregressor__max_leaves": range(5, 20, 4),
    "xgbregressor__learning_rate": [0.001, 0.01, 0.1, 0.2]
}
xgb_search = RandomizedSearchCV(
    xgb_pipeline,
    param_grid_xgb,
    scoring="neg_mean_squared_log_error",
    n_iter=20, cv=5, verbose=1)

xgb_search.fit(X_train, y_train)

# Model performance
xgb_performance = get_model_metrics("XGB", y_test, xgb_search.predict(X_test))
model_scores = pd.concat([model_scores, xgb_performance])

print_best_parameters(xgb_search, X_test, y_test)

Fitting 5 folds for each of 20 candidates, totalling 100 fits

RMSE_log:  0.019571415107297825
Best logreg parameters: 
 {'xgbregressor__n_estimators': 100, 'xgbregressor__max_leaves': 9, 'xgbregressor__max_depth': 7, 'xgbregressor__learning_rate': 0.2}


<a id="gradient"></a>
### 1.3&nbsp; Gradient Boosting Regressor

In [9]:
# Make pipeline
gradboost_pipeline = make_pipeline(
    preprocessing_pipe,
    GradientBoostingRegressor()
)

# Perform hyper parameter search
param_grid_gradboost = {
    "gradientboostingregressor__loss": ["squared_error", "absolute_error", "huber", "quantile"],
    "gradientboostingregressor__learning_rate": [0.001, 0.01, 0.1, 0.2],
    "gradientboostingregressor__n_estimators": range(50, 500, 50),
    "gradientboostingregressor__subsample": [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
}
gradboost_search = RandomizedSearchCV(
    gradboost_pipeline,
    param_grid_gradboost,
    scoring="neg_mean_squared_log_error",
    n_iter=20, cv=5, verbose=1)

gradboost_search.fit(X_train, y_train)

# Model performance
gradboost_performance = get_model_metrics("Gradient Boosting", y_test, gradboost_search.predict(X_test))
model_scores = pd.concat([model_scores, gradboost_performance])
print_best_parameters(gradboost_search, X_test, y_test)

Fitting 5 folds for each of 20 candidates, totalling 100 fits

RMSE_log:  0.015388725753474447
Best logreg parameters: 
 {'gradientboostingregressor__subsample': 0.8, 'gradientboostingregressor__n_estimators': 450, 'gradientboostingregressor__loss': 'huber', 'gradientboostingregressor__learning_rate': 0.1}


<a id="poisson"></a>
### 1.4&nbsp; Poisson Regressor

In [10]:
# Make pipeline
poisson_pipeline = make_pipeline(
    preprocessing_pipe,
    PoissonRegressor()
)

# Perform hyper parameter search
param_grid_poisson = {
    "poissonregressor__solver": ["lbfgs", "newton-cholesky"],
    "poissonregressor__max_iter": range(50, 500, 50),
    "poissonregressor__fit_intercept": [True, False],
    "poissonregressor__tol": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100] 
}
poisson_search = RandomizedSearchCV(
    poisson_pipeline,
    param_grid_poisson,
    scoring="neg_mean_squared_log_error",
    n_iter=20, cv=5, verbose=1)

poisson_search.fit(X_train, y_train)

# Model performance
poisson_performance = get_model_metrics("Poisson", y_test, poisson_search.predict(X_test))
model_scores = pd.concat([model_scores, poisson_performance])
print_best_parameters(poisson_search, X_test, y_test)

Fitting 5 folds for each of 20 candidates, totalling 100 fits

RMSE_log:  0.022652463618808415
Best logreg parameters: 
 {'poissonregressor__tol': 1, 'poissonregressor__solver': 'lbfgs', 'poissonregressor__max_iter': 50, 'poissonregressor__fit_intercept': True}


___
<a id="selection"></a>
# 2&nbsp; Model Selection

<a id="scores"></a>
### 2.1&nbsp; Model Scores

In [11]:
model_scores.sort_values("RMSE_log")

,R2,MAE,MAPE,MSE,RMSE_log
Gradient Boosting,0.91,15349.17,0.09,691187334.36,0.02
XGB,0.91,16229.09,0.10,659603587.79,0.02
Poisson,0.91,17028.93,0.11,686603628.92,0.02


According to the test accuracies in `model_scores`, the best performing models were the Logistic Regression, the Random Forest, the Support Vector Machine and the XGBoost models. 

When sorting by the root mean squared log (RMSE_log), the Gradient Boosting model performs slightly better than the rest.

<a id="voting"></a>
### 2.2&nbsp; Voting Regressor

Choose the best performing models and use them with a [VotingRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingRegressor.html#sklearn.ensemble.VotingRegressor) to potentially further increase performance.

In [12]:
# Re-establish the best regressors
model_xgb = XGBRegressor(n_estimators=200, max_leaves=9, max_depth=11, learning_rate=0.1)
model_gradboost = GradientBoostingRegressor(subsample=0.8, n_estimators=250, loss='huber', learning_rate=0.1)
model_poisson = PoissonRegressor(tol=0.0001, solver='lbfgs', max_iter=50, fit_intercept=True)
# Take the best performing hyperparameters from the GridSearches above

# Create the voting regressor
voting_regressor = VotingRegressor(
    estimators=[
        ('XGB', model_xgb), 
        ('GradientBoosting', model_gradboost),
        ('Poisson', model_poisson)])

# Preprocessing pipeline + voting regressor
full_pipe_voting = make_pipeline(
    preprocessing_pipe,
    voting_regressor)

full_pipe_voting.fit(X_train, y_train)

VotingRegressor(estimators=[('XGB',
                             XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate...
                                          max_cat_to_onehot=None,
                                          max_delta_step=None, max_depth=11,
                                          max_leaves=9, min_child_weight=None,
                                          missing=nan,
                                          monotone_constraints=None,
                                          multi_strategy=None, n_estimators=200,
                                          n_jobs=None, num_parallel_tree=None,
                                          random_state=None, ...)),
                            ('GradientBoosting',
                             GradientBoostingRegressor(loss='huber',
                                                       n_estimators=250,
                                                       subsample=0.8)),
                            ('Poisson', PoissonRegressor(max_iter=50))])

In [14]:
# Estimate test accuracies
for regressor, label in zip(
    [model_xgb, model_gradboost, model_poisson, voting_regressor], 
    ['XGB', 'GradientBoosting', 'Poisson', 'Ensemble']):
    
    # Full pipeline: preprocessor + regressor
    full_pipe = make_pipeline(
        preprocessing_pipe,
        regressor)
    
    # Fit models & calculate test accuracies
    scores = cross_val_score(full_pipe, X_test, y_test, cv=5)
    print("Accuracy: %0.3f (+/- %0.3f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.837 (+/- 0.045) [XGB]
Accuracy: 0.842 (+/- 0.037) [GradientBoosting]
Accuracy: 0.854 (+/- 0.052) [Poisson]
Accuracy: 0.865 (+/- 0.033) [Ensemble]


The Voting Regressor has a slightly worse performance than the single regressors. Too bad! 

We'll use the **Gradient Boosting model** then, as this one performed best during training.

___
<a id="prediction"></a>
# 3&nbsp; Make Prediction

#### Train model on complete train dataset

Note that in Section 2 we used `RandomSearchCV` to speed things up. Here we use `GridSearchCV` to find the optimal hyper parameters. 

Also note that in this section we use the complete dataset (`X` and `y`) to train the model. We do not have to tune hyper parameters any longer and want to go for the final, best model version. We'll use that best version to make a prediction on the validation dataset from Kaggle.

In [17]:
# Make pipeline
gradboost_pipeline = make_pipeline(
    preprocessing_pipe,
    GradientBoostingRegressor()
)

# Perform hyper parameter search
param_grid_gradboost = {
    "gradientboostingregressor__loss": ["squared_error", "absolute_error", "huber", "quantile"],
    "gradientboostingregressor__learning_rate": [0.001, 0.01, 0.1, 0.2],
    "gradientboostingregressor__n_estimators": range(50, 500, 50),
    "gradientboostingregressor__subsample": [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
}
gradboost_search = GridSearchCV(
    gradboost_pipeline,
    param_grid_gradboost,
    scoring="neg_mean_squared_log_error",
    cv=5, verbose=1)

gradboost_search.fit(X, y) # use the whole dataset

Fitting 5 folds for each of 864 candidates, totalling 4320 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('num_pipe',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2...
                                        GradientBoostingRegressor())]),
             param_grid={'gradientboostingregressor__learning_rate': [0.001,
                                                                      0.01, 0.1,
                                                                      0.2],
                         'gradientboostingregressor__loss': ['squared_error',
                                                             'absolute_error',
                                                             'huber',
                                                             'quantile'],
                         'gradientboostingregressor__n_estimators': range(50, 500, 50),
                         'gradientboostingregressor__subsample': [0.0, 0.2, 0.4,
                                                                  0.6, 0.8,
                                                                  1.0]},
             scoring='neg_mean_squared_log_error', verbose=1)

In [18]:
# Model performance
print_best_parameters(gradboost_search, X_test, y_test)
get_model_metrics("Gradient Boosting", y_test, gradboost_search.predict(X_test))


RMSE_log:  0.004069175223334085
Best logreg parameters: 
 {'gradientboostingregressor__learning_rate': 0.1, 'gradientboostingregressor__loss': 'huber', 'gradientboostingregressor__n_estimators': 350, 'gradientboostingregressor__subsample': 1.0}


,R2,MAE,MAPE,MSE,RMSE_log
Gradient Boosting,0.98,7329.62,0.04,164279200.11,0.00


#### Read in test data

In [19]:
X_val = pd.read_csv("data/test.csv")
IDs = X_val.pop("Id") # The ID column is for identifying the predicted labels

# Are the columns the same as in the train/test sets?
set(X_val.columns) == set(X_train.columns) == set(X_test.columns)

True

#### Make prediction

In [20]:
# Predict labels of validation set
price_predictions = gradboost_search.predict(X_val)

predicted_labels = pd.DataFrame({
    "Id": IDs,
    "SalePrice": price_predictions
})

# Equal length of dataset and predictions?
print(predicted_labels.shape[0] == X_val.shape[0])

# Export predicted labels with their IDs
predicted_labels.to_csv("data/submission.csv", index=False)

True


When uploading our `submission.csv` file based on the `GradientBoostingRegressor` to the Kaggle competition page, we got a public score of 0.13378. That's place 1384 out of roughly 5000 competitors. 

A submission using the `XGBRegressor` in a similar manner got us a public score of 0.13143, ranking place 1213 on the leaderboard.

Not bad!